In [174]:
import pandas as pd
import os
import gc
import pickle

from time import sleep

import gspread_dataframe as gs_df

#os.chdir("../")

# INITIALIZE DRIVER
from selenium import webdriver
from sapiens import LoginPage, KEYS, Page_sapiens
import functions


% reload_ext autoreload
% autoreload 2

### Loga Driver no Sapiens

In [173]:
driver = webdriver.Firefox()
driver = LoginPage(driver).login('33831640807', 'Fer465023@ur_')    

### Criar o Data Frame para exportar os dados

In [180]:
database = pd.read_csv("registros_cadin.csv")

In [181]:
database = database.dropna(subset=["CPF"])

In [182]:
registros = {row[1]["CPF"]:{k:v for k,v in row[1].items()} for row in df.iterrows()}

In [183]:
len(registros)        

6217

In [175]:
sapiens = Page_sapiens(driver, registros)

### Leitura dos Dados Brutos na Planilha

In [5]:
gc = functions.authenticate_google("files/anatel.json")
sht = gc.open(title='RC')

In [6]:
registros = sht.worksheet('Registros')
registros_rf = sht.worksheet('Registros_RF')

In [171]:
#lista_CPF = registros.col_values(1)[1:]

lista_cpf = df["CPF"]

In [179]:
counter = 0

for cpf in lista_cpf:
    
    if len(cpf) in (12, 13, 14):
        
        while len(cpf) < 14:
            
            cpf = '0' + str(cpf)
    
    while len(cpf) < 11:

        cpf = '0' + str(cpf)

    registro = sapiens.get_registro(cpf)    
   
    if not registro:
    
        sapiens.pesquisa_dados(cpf)
            
        registro = sapiens.get_registro(cpf)
        
        counter += 1
        
   
    if not (df["CPF"] == registro.get(cpf)).any():
        
        df.append(pd.Series(registro), ignore_index=True)
        
    if counter > 100:
        
        df = df.fillna("")
        
        #df.to_csv("registros_cadin.csv", index=False)
        
        driver = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
        
        driver = LoginPage(driver).login('33831640807', 'Fer465023@ur_')
        
        sapiens.reset_driver(driver)        
        
        counter = 0  

KeyboardInterrupt: 

In [59]:
df = df.fillna("")

In [60]:
df["Endereço_Completo"] = df.apply(lambda row: " ".join([str(row["Logradouro"]), 
                                                         str(row["Número"]), 
                                                         str(row["Complemento"]),
                                                         str(row["Bairro"]), "\n", "CEP:", 
                                                         str(row["Cep"]), 
                                                         str(row["Cidade"])]
                                                       ), axis=1
                                  )

In [65]:
writer = pd.ExcelWriter("files/planilha_obito.xlsx")
df.to_excel(writer, "consulta_RF", index=False, encoding='utf-8')
writer.save()

In [25]:
#gs_df.set_with_dataframe(registros_rf, df, row=142, include_column_header=False)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Nome"]), row=2, col=9, include_column_header=False)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Endereço_Completo"]), row=2, col=12, include_column_header=False)


APIError: {
  "error": {
    "code": 401,
    "message": "Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",
    "status": "UNAUTHENTICATED"
  }
}
